# Nimbus prediction notebook 

In [1]:
# import required packages
import warnings
warnings.simplefilter("ignore")
import os
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from nimbus_inference.nimbus import Nimbus, prep_naming_convention
from nimbus_inference.utils import MultiplexDataset
from alpineer import io_utils
#from ark.utils import example_dataset
from nimbus_inference.viewer_widget import NimbusViewer

## 0: Set root directory and download example dataset
Here we are using the example data located in `/data/example_dataset/input_data`. To modify this notebook to run using your own data, simply change `base_dir` to point to your own sub-directory within the data folder. Set `base_dir`, the path to all of your imaging data (i.e. multiplexed images and segmentation masks). Subdirectory `nimbus_output` will contain all of the data generated by this notebook. In the following, we expect this folder structure, with `fov_1` and `fov_2` either being folders of individual channel images or `.ome.tiff` files that contain all channels in a single file.
```bash
|-- base_dir
|   |-- image_data
|   |   |-- fov_1
|   |   |-- fov_2
|   |-- segmentation
|   |   |-- deepcell_output
|   |-- nimbus_output
```

In [2]:
# set up the base directory
base_dir = os.path.normpath("/Volumes/Gut_Project/pheno_benchmark/Maps_data/cHL_2_MIBI")
output_dir = os.path.normpath("/Users/margotchazotte/Documents/uni/PhD/pheno_benchmark/nimbus/nimbus_chl2")


If you would like to test Nimbus with an example dataset, run the cell below. It will download a dataset consisting of 10 FOVs with 22 channels. You may find more information about the example dataset in the [ark-analysis README](https://github.com/angelolab/ark-analysis/blob/bc6685050dfbef4607874fbbadebd4289251c173/README.md#example-dataset). If you want to use your own data, skip the cell below


## 1: set file paths and parameters

### All data, images, files, etc. must be placed in the 'data' directory, and referenced via '../data/path_to_your_data'


In [3]:
# set up file paths
tiff_dir = os.path.join(base_dir, "raw_image")
deepcell_output_dir = os.path.join(base_dir, "segmentation", "masks")
nimbus_output_dir = os.path.join(output_dir, "it5/nimbus_output")

# Create nimbus output directory
os.makedirs(nimbus_output_dir, exist_ok=True)

# Check if paths exist
io_utils.validate_paths([base_dir, tiff_dir, deepcell_output_dir, nimbus_output_dir])

## 2: Set up input paths and the naming convention for the segmentation data
Store names of channels to exclude in the list below. Either predict all FOVs or specify manually the ones you want to apply Nimbus on.

In [4]:
# define the channels to include
include_channels = ['CD14', 'HLA1', 'GATA3', 'HLA-DR', 'CXCR5', 'CD15', 'Pax-5', 'anti-H2AX (pS139)', 'CD45', 
                    'RORgT', 'CD86', 'CD123', 'CD28', 'Granzyme B', 'CD68', 'Tbet', 'CD163', 'FoxP3', 'CD11c', 
                    'Tox', 'CD25', 'TCRgd', 'PD-1', 'CD45RO', 'CD30', 'CD8', 'CD4', 'SLP-76', 'Histone H3', 
                    'CD45RA', 'pSLP-76', 'CD161', 'CD153 (CD30L)', 'Lag3', 'IL-10', 'CD16', 'Na-K ATPase', 'B2-Microglobulin', 
                    'CD57', 'CD20', 'TIM3', 'Ki-67', 'PD-L1', 'CD56', 'CD3']

# either get all fovs in the folder...
fov_names = os.listdir(tiff_dir)
# ... or optionally, select a specific set of fovs manually
# fovs = ["fov0", "fov1"]

# make sure to filter paths out that don't lead to FoVs, e.g. .DS_Store files.
fov_names = [fov_name for fov_name in fov_names if not fov_name.startswith(".")] 

# construct paths for fovs
fov_paths = [os.path.join(tiff_dir, fov_name) for fov_name in fov_names]

In [5]:
fov_paths , fov_names

(['/Volumes/Gut_Project/pheno_benchmark/Maps_data/cHL_2_MIBI/raw_image/fov_2',
  '/Volumes/Gut_Project/pheno_benchmark/Maps_data/cHL_2_MIBI/raw_image/fov_5',
  '/Volumes/Gut_Project/pheno_benchmark/Maps_data/cHL_2_MIBI/raw_image/fov_4',
  '/Volumes/Gut_Project/pheno_benchmark/Maps_data/cHL_2_MIBI/raw_image/fov_3',
  '/Volumes/Gut_Project/pheno_benchmark/Maps_data/cHL_2_MIBI/raw_image/fov_6',
  '/Volumes/Gut_Project/pheno_benchmark/Maps_data/cHL_2_MIBI/raw_image/fov_1'],
 ['fov_2', 'fov_5', 'fov_4', 'fov_3', 'fov_6', 'fov_1'])

Define the naming convention for the segmentation data in function `segmentation_naming_convention`, that maps the `fov_name` to the path of the associated segmentation output. The below function `prep_deepcell_naming_convention` assumes that all segmentation outputs are stored in one folder, with the `fov_name` as the prefix and `_whole_cell.tiff` as the suffix, as shown below in the visualization of the folder structure. If this does not apply to your data, you have to define a function `segmentation_naming_convention` that takes an element from `fov_paths` and returns a valid path to the segmentation label map you want to use for that fov.

```
|-- base_dir
|   |-- image_data
|   |   |-- fov_1
|   |   |-- fov_2
|   |-- segmentation
|   |   |-- deepcell_output
|   |   |   |-- fov_1_whole_cell.tiff
|   |   |   |-- fov_2_whole_cell.tiff
|   |-- nimbus_output
```

In [6]:
def prep_naming_convention(deepcell_output_dir):
    """Prepares the naming convention for the segmentation data produced with the DeepCell library.

    Args:
        deepcell_output_dir (str): path to directory where segmentation data is saved
    Returns:
        function: function that returns the path to the
            segmentation data for a given fov
    """

    def segmentation_naming_convention(fov_path):
        """Prepares the path to the segmentation data for a given fov

        Args:
            fov_path (str): path to fov
        Returns:
            str: paths to segmentation fovs
        """
        fov_name = os.path.basename(fov_path).replace(".ome.tiff", "")
        return os.path.join(deepcell_output_dir, fov_name + ".tif")

    return segmentation_naming_convention

In [7]:
# Prepare segmentation naming convention that maps a fov_path to the according segmentation label map
segmentation_naming_convention = prep_naming_convention(deepcell_output_dir)

# test segmentation_naming_convention
if os.path.exists(segmentation_naming_convention(fov_paths[0])):
    print("Segmentation data exists for fov 0 and naming convention is correct")
else:
    print("Segmentation data does not exist for fov 0 or naming convention is incorrect")

Segmentation data exists for fov 0 and naming convention is correct


Next we will use the `MultiplexDataset` class to abstract away differences in data representation. The class takes `fov_paths`, `segmentation_naming_convention` and a `suffix` and provides methods `.get_channel(fov, channel)` and `.get_segmentation(fov)` to access the data. The `suffix` is used to filter out files that do not end with the specified suffix. When you use `.ome.tiff` files make sure to set the suffix to `.ome.tiff`, otherwise the ViewerWidget won't be able to display the images.

In [8]:
dataset = MultiplexDataset(
    fov_paths=fov_paths,
    suffix=".tiff", # or .png, .jpg, .jpeg, .tif or .ome.tiff
    include_channels=include_channels,
    segmentation_naming_convention=segmentation_naming_convention,
)

All inputs are valid


## 3: Load model and initialize Nimbus application
The following code initializes the Nimbus application and loads the model checkpoint. The model was trained on a diverse set of tissues, protein markers, imaging platforms and cell types and doesn't need re-training. If you want to use the model on a machine without GPU, set `test_time_aug=False` to speed up inference. If you run it on a laptop GPU and run into out-of-memory errors, consider reducing the `batch_size` to 1 and the `input_shape` to `[512,512]`.

In [9]:
nimbus = Nimbus(
    dataset=dataset,
    output_dir=nimbus_output_dir,
    save_predictions=True,
    batch_size=4,
    test_time_aug=False,
    input_shape=[1024,1024],
    device="mps",
)

# check if all inputs are valid
nimbus.check_inputs()

All inputs are valid.


## 4: Prepare normalization dictionary 
The next step is to iterate through all the fovs and calculate the 0.999 marker expression quantile for each marker individually. This is used for normalizing the marker expressions prior to predicting marker confidence scores with our model. You can set `n_subset` to estimate the quantiles on a small subset of the data and you can set `multiprocessing=True` to speed up computation.

In [10]:
nimbus.prepare_normalization_dict(
    quantile=0.999,
    n_subset=50,
    clip_values=(0, 2),
    multiprocessing=True,
    overwrite=True
)

Iterate over fovs...


## 5: Make predictions with the model
Nimbus will iterate through your samples and store predictions and a file named `nimbus_cell_table.csv` that contains the mean-per-cell predicted marker confidence scores in the sub-directory called `nimbus_output`.

In [11]:
cell_table = nimbus.predict_fovs()

Available GPUs:  0
Predictions will be saved in /Users/margotchazotte/Documents/uni/PhD/pheno_benchmark/nimbus/nimbus_chl2/it5/nimbus_output
Iterating through fovs will take a while...
Predicting /Volumes/Gut_Project/pheno_benchmark/Maps_data/cHL_2_MIBI/raw_image/fov_2...


  0%|          | 0/45 [00:00<?, ?it/s]

Checking for updated model checkpoints on HuggingFace Hub...
Using existing checkpoint: /Users/margotchazotte/anaconda3/envs/nimbus-env/lib/python3.9/site-packages/nimbus_inference/assets/V1.pt
Loaded weights from /Users/margotchazotte/anaconda3/envs/nimbus-env/lib/python3.9/site-packages/nimbus_inference/assets/V1.pt


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Predicting /Volumes/Gut_Project/pheno_benchmark/Maps_data/cHL_2_MIBI/raw_image/fov_1...


  0%|          | 0/45 [00:00<?, ?it/s]

Checking for updated model checkpoints on HuggingFace Hub...
Using existing checkpoint: /Users/margotchazotte/anaconda3/envs/nimbus-env/lib/python3.9/site-packages/nimbus_inference/assets/V1.pt
Loaded weights from /Users/margotchazotte/anaconda3/envs/nimbus-env/lib/python3.9/site-packages/nimbus_inference/assets/V1.pt


In [12]:
cell_table

,label,fov,CD14,HLA1,GATA3,HLA-DR,CXCR5,CD15,Pax-5,anti-H2AX (pS139),...,CD16,Na-K ATPase,B2-Microglobulin,CD57,CD20,TIM3,Ki-67,PD-L1,CD56,CD3
0,1,fov_2,0.039919,0.087493,0.051003,0.045590,0.028922,0.034429,0.038947,0.038585,...,0.171313,0.034250,0.109506,0.036893,0.036427,0.055426,0.028883,0.067354,0.038738,0.058574
1,2,fov_2,0.029231,0.033596,0.029131,0.225328,0.028517,0.029178,0.029115,0.029207,...,0.029103,0.030195,0.030611,0.029445,0.029117,0.029044,0.029086,0.029872,0.029184,0.071514
2,3,fov_2,0.026375,0.024071,0.024507,0.025163,0.024605,0.024446,0.024308,0.025178,...,0.025346,0.024730,0.023891,0.024705,0.024735,0.024493,0.024510,0.024782,0.024784,0.023809
3,4,fov_2,0.084863,0.032426,0.029073,0.026982,0.025440,0.025692,0.026536,0.024900,...,0.026638,0.026555,0.035759,0.026783,0.024438,0.024147,0.031820,0.024422,0.023969,0.051328
4,5,fov_2,0.023630,0.022745,0.022437,0.023267,0.022453,0.022487,0.022281,0.022866,...,0.022939,0.022768,0.022921,0.022477,0.022389,0.022426,0.022284,0.022546,0.022562,0.022617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233911,38111,fov_1,0.025772,0.024213,0.025211,0.025901,0.025753,0.025904,0.026019,0.025905,...,0.026068,0.024831,0.023506,0.025901,0.025930,0.026405,0.025904,0.025606,0.025908,0.026660
233912,38112,fov_1,0.032393,0.030048,0.040208,0.037012,0.032057,0.035415,0.043513,0.035707,...,0.036273,0.031679,0.027473,0.038401,0.035550,0.034718,0.035832,0.036420,0.035702,0.062492
233913,38113,fov_1,0.026574,0.030913,0.037293,0.032365,0.028923,0.026495,0.028705,0.026615,...,0.026540,0.049049,0.026677,0.027375,0.026649,0.027593,0.026791,0.031723,0.026602,0.090237
233914,38114,fov_1,0.029038,0.040055,0.069663,0.037540,0.027125,0.032406,0.034574,0.032506,...,0.032687,0.038578,0.037675,0.032224,0.032361,0.029329,0.032614,0.030189,0.032490,0.052991


In [13]:
cell_table.to_csv(os.path.join(nimbus_output_dir, "nimbus_cell_table.csv"), index=False)

## 6: View multiplexed channels and Nimbus predictions side-by-side
Select an FOV and one marker image per channel to inspect the imaging data and associated Nimbus predictions